<a target="_blank" href="../cluster" style="font-size:20px">All Applications (YARN)</a>

# Домашнее задание

Будем использовать логи прослушивания музыкальных исполнителей в сервисе Яндекс.Музыка.

Файл `events.csv` содержит записи вида `Пользователь,Исполнитель,Число прослушиваний,Число пропусков`:
```csv
userId,artistId,plays,skips
0,335,1,0
0,708,1,0
0,710,2,1
0,815,1,1
```

Вам необходимо проделать следующее:
1. **Оставьте в данных только тех пользователей, для которых сумма plays строго больше 1000. Сколько таких пользователей?**
2. **В отфильтрованных на первом шаге данных найдите 5 самых популярных по числу пользователей исполнителей (идентификаторы).**

Детали:
1. Давайте считать, что список прослушиваний одного пользователя всегда помещается в память.

Решение сохраните в файл `result.json`. Пример содержимого файла:

```json
{
    "q1": 123,
    "q2": [
        4,
        5,
        6,
        7,
        8
    ]
}```

In [1]:
# пример содержимого файла
! head -n 5 yandex_music/events.csv

userId,artistId,plays,skips
0,335,1,0
0,708,1,0
0,710,2,1
0,815,1,1


In [1]:
! cat yandex_music/events.csv | tail -n +2 > yandex_music/events_no_header.csv

In [2]:
! ls yandex_music

artists.jsonl  events.csv  events_no_header.csv  README.txt


In [3]:
# копируем файлы в HDFS
! hadoop fs -copyFromLocal yandex_music /
! hadoop fs -ls -h /yandex_music

Found 4 items
-rw-r--r--   1 jovyan supergroup        254 2021-08-11 13:59 /yandex_music/README.txt
-rw-r--r--   1 jovyan supergroup      3.7 M 2021-08-11 13:59 /yandex_music/artists.jsonl
-rw-r--r--   1 jovyan supergroup     47.6 M 2021-08-11 13:59 /yandex_music/events.csv
-rw-r--r--   1 jovyan supergroup     47.6 M 2021-08-11 13:59 /yandex_music/events_no_header.csv


In [ ]:
# ТУТ ВАШ КОД

## Тестируем локально

In [4]:
%%file mapper.py
import sys

for line in sys.stdin:
    userId, _, plays, _ = line.split(',')
    print(userId + "\t" + plays)

Overwriting mapper.py


In [8]:
%%file reducer.py
import sys

prev_key = None
count = 0
for line in sys.stdin:  # stream is sorted by key
    key, value = line.split("\t")
    
    if prev_key is not None and key != prev_key:
        # new key in stream, dump previous
        if count > 2000:
            print(prev_key)
        count = 0
    
    count += int(value)
    prev_key = key

# dump last key
if count > 2000:
    print(prev_key)

Overwriting reducer.py


In [9]:
%%time
! cat yandex_music/events_no_header.csv | python ./mapper.py | python ./reducer.py > step1.txt

CPU times: user 68.6 ms, sys: 18.3 ms, total: 86.9 ms
Wall time: 5.03 s


In [10]:
! cat step1.txt | wc -l

1705


In [68]:
! hadoop fs -rm -r /music-count

! mapred streaming \
  -input /yandex_music/events_no_header.csv \
  -output /music-count \
  -mapper "/opt/conda/bin/python3.6 mapper.py" \
  -reducer "/opt/conda/bin/python3.6 reducer.py" \
  -file mapper.py \
  -file reducer.py \

Deleted /music-count
2021-08-11 11:38:24,119 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper.py, reducer.py] [/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar] /tmp/streamjob5952599561469515453.jar tmpDir=null
2021-08-11 11:38:24,915 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2021-08-11 11:38:25,072 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2021-08-11 11:38:25,251 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/jovyan/.staging/job_1628676009592_0004
2021-08-11 11:38:25,492 INFO mapred.FileInputFormat: Total input files to process : 1
2021-08-11 11:38:25,522 INFO mapreduce.JobSubmitter: number of splits:3
2021-08-11 11:38:25,616 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1628676009592_0004
2021-08-11 11:38:25,616 INFO mapreduce.Job

In [77]:
! hadoop fs -cat /music-count/part-00000 | wc -l

3117


In [82]:
%%file mapper2.py
import sys
with open('step1.txt', 'r') as myfile:
    data = myfile.read().split()
    
for line in sys.stdin:
    userId, artistId, _, _ = line.split(',')
    if userId in data:
        print(artistId + "\t" + "1")

Overwriting mapper2.py


In [83]:
%%file reducer2.py
import sys

prev_key = None
count = 0
for line in sys.stdin:  # stream is sorted by key
    key, value = line.split("\t")
    
    if prev_key is not None and key != prev_key:
        # new key in stream, dump previous
        print(prev_key + "\t" + str(count))
        count = 0
    
    count += int(value)
    prev_key = key

# dump last key
print(prev_key + "\t" + str(count))

Overwriting reducer2.py


In [84]:
! hadoop fs -rm -r /music-author

! mapred streaming \
  -input /yandex_music/events_no_header.csv \
  -output /music-author \
  -mapper "/opt/conda/bin/python3.6 mapper2.py" \
  -reducer "/opt/conda/bin/python3.6 reducer2.py" \
  -file mapper2.py \
  -file reducer2.py \
  -file step1.txt \

Deleted /music-author
2021-08-11 11:51:30,720 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper2.py, reducer2.py, step1.txt] [/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar] /tmp/streamjob8797366671545227778.jar tmpDir=null
2021-08-11 11:51:31,510 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2021-08-11 11:51:31,654 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2021-08-11 11:51:31,828 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/jovyan/.staging/job_1628676009592_0008
2021-08-11 11:51:32,059 INFO mapred.FileInputFormat: Total input files to process : 1
2021-08-11 11:51:32,495 INFO mapreduce.JobSubmitter: number of splits:3
2021-08-11 11:51:32,581 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1628676009592_0008
2021-08-11 11:51:32,581 INFO

In [105]:
! hadoop fs -cat /music-author/part-00000 | sort -k2 -n -r | head -n 5

11368	2574
3629	2286
259	2208
44148	2161
23524	2110
sort: write failed: 'standard output': Broken pipe
sort: write error


In [106]:
%%file result.json
{
    "q1": 3117,
    "q2": [
        11368,
        3629,
        259,
        44148,
        23524
    ]
}

Writing result.json


In [107]:
!cat result.json

{
    "q1": 3117,
    "q2": [
        11368,
        3629,
        259,
        44148,
        23524
    ]
}